# PDF Text Extractor Utility

> Extracts unstructured text from PDF Files. This is intentionally bare bones and does not attempt to extract tables, images, or other data from PDFs. It is intended to be used as a preprocessing step for other NLP tasks.

In [1]:
#| default_exp utils.pdf_text_extractor

In [4]:
#| export

'''from pypdf import PdfReader
import dataclasses
from typing import List, Tuple, Any, Dict
'''

In [5]:
#| export
'''
@dataclasses.dataclass
class PdfDoc:
    """A class that provides structure for full text papers specified under the JATS 'nxml' format."""

    # The location of the file
    url: str

    # The identifier of the paper
    ft_id: str

    # Plain text of the paper's contents
    text: str
      
    def __init__(self, ft_id, url):
        self.ft_id = ft_id

        reader = PdfReader(url)
        number_of_pages = len(reader.pages)
        text = ''
        for p in reader.pages:
            text += p.extract_text()
        self.text = text
'''